# Password Checker II

Continuing the build of the checker, we have our `pwned_api_check(password)` that only converts our password to SHA-1, but we need to send it to the password API.

1. We only need the first five chars of our hashed password.

In [3]:
import requests
import hashlib # for SHA-1 hashing


def request_api_data(query_char):
    url = 'https://api.pwnedpasswords.com/range/' + query_char
    res = requests.get(url)
    if res.status_code != 200:
      raise RuntimeError(f'Error fetching: {res.status_code}, check the API and try again')
    return res

def pwned_api_check(password):
    sha1password = hashlib.sha1(password.encode('utf-8')).hexdigest().upper()
    first5_char, tail = sha1password[:5], sha1password[5:]
    response = request_api_data(first5_char)
    print(first5_char, tail)
    return response
    print(first5_char, tail)
    
pwned_api_check('123')

40BD0 01563085FC35165329EA1FF5C5ECBDBBEEF


<Response [200]>

We're still unsure of the type of data we're receiving. We'll need to create another function

In [4]:
import requests
import hashlib # for SHA-1 hashing


def request_api_data(query_char):
    url = 'https://api.pwnedpasswords.com/range/' + query_char
    res = requests.get(url)
    if res.status_code != 200:
      raise RuntimeError(f'Error fetching: {res.status_code}, check the API and try again')
    return res
  
def read_res(response):
  print(response.text)

def pwned_api_check(password):
    sha1password = hashlib.sha1(password.encode('utf-8')).hexdigest().upper()
    first5_char, tail = sha1password[:5], sha1password[5:]
    response = request_api_data(first5_char)
    print(first5_char, tail)
    return read_res(response)
    print(first5_char, tail)
    
pwned_api_check('123')

40BD0 01563085FC35165329EA1FF5C5ECBDBBEEF
005BA21B62A7D0DA863D9858C6239FB8E3A:1
00684A10635394CC9D4524FC8C41580159E:1
00F6F1EFBEC383A639129970B019A4FB1A9:3
01563085FC35165329EA1FF5C5ECBDBBEEF:196
018A46B2D5E8776A77CAF8C912265DDFD11:3
01946E1895C216F090C748DDCA927C30760:1
01E0902635AE8545FB1D6F92C9B9ABD6511:1
02AFCF3E1B1A6475C674B12EB8FE76127E9:4
02E15B3EB64C929774FD029643182A86F73:1
0402B48DE3DCE1BB07456F6EE2FDD19B551:3
040F6CFD53A6C428627BC9FB8F75AEB56FE:1
0440AC9AA0AE7DDBED11EA139939D326E1C:3
048645E380FB5A5F6156E487477A6B63A28:1
0521DCC6675EBD7DA94968C401923560705:2
0556D127736BB8456D8B182AF51A2E584DA:1
055EF5D985576DCFEFFDCBF9CAA6847A3A4:3
056BE57C0C8DC043B2AF879BE824725F66D:3
05DBEF99250DF672C48962B193693A0C6C9:2
06305F033B219CEE7D2BF247956FBE6A422:1
0658AF19B64CC50E7ACB4B39D84C58AE836:1
06654999297DF4791050E04D884A24500A4:1
06657E28CCF3E94745F0BC16C23C69D9FE0:1
06BD2FA53BFB27C77091B260B3B4E507D7D:1
0700E850D28319CDB10F0D31BBFB911E9B1:2
07EA3763DDA9C2524CF7A94914230350490:1
0860CA

The response is a bunch of hashed passwords. We get all the responses that match the beginning of our hashed password. THe number after the colon tells us how many times the password has been hacked. Ideally, we can grab the count and see if any of it matches with our tail.

So, we check the tail and print out the count. Instead of having a `read_res` function, we'll have a `get_password_leaks_count` function

In [13]:
import requests
import hashlib # for SHA-1 hashing


def request_api_data(query_char):
    url = 'https://api.pwnedpasswords.com/range/' + query_char
    res = requests.get(url)
    if res.status_code != 200:
      raise RuntimeError(f'Error fetching: {res.status_code}, check the API and try again')
    return res
  
def get_password_leaks_count(hashes, hash_to_check):
  hashes = (line.split(':') for line in hashes.text.splitlines())
  for h, count in hashes:
    print(h, count)

def pwned_api_check(password):
    sha1password = hashlib.sha1(password.encode('utf-8')).hexdigest().upper()
    first5_char, tail = sha1password[:5], sha1password[5:]
    response = request_api_data(first5_char)
    print(response)
    return get_password_leaks_count(response, tail)
    
    
pwned_api_check('123')

<Response [200]>
005BA21B62A7D0DA863D9858C6239FB8E3A 1
00684A10635394CC9D4524FC8C41580159E 1
00F6F1EFBEC383A639129970B019A4FB1A9 3
01563085FC35165329EA1FF5C5ECBDBBEEF 196
018A46B2D5E8776A77CAF8C912265DDFD11 3
01946E1895C216F090C748DDCA927C30760 1
01E0902635AE8545FB1D6F92C9B9ABD6511 1
02AFCF3E1B1A6475C674B12EB8FE76127E9 4
02E15B3EB64C929774FD029643182A86F73 1
0402B48DE3DCE1BB07456F6EE2FDD19B551 3
040F6CFD53A6C428627BC9FB8F75AEB56FE 1
0440AC9AA0AE7DDBED11EA139939D326E1C 3
048645E380FB5A5F6156E487477A6B63A28 1
0521DCC6675EBD7DA94968C401923560705 2
0556D127736BB8456D8B182AF51A2E584DA 1
055EF5D985576DCFEFFDCBF9CAA6847A3A4 3
056BE57C0C8DC043B2AF879BE824725F66D 3
05DBEF99250DF672C48962B193693A0C6C9 2
06305F033B219CEE7D2BF247956FBE6A422 1
0658AF19B64CC50E7ACB4B39D84C58AE836 1
06654999297DF4791050E04D884A24500A4 1
06657E28CCF3E94745F0BC16C23C69D9FE0 1
06BD2FA53BFB27C77091B260B3B4E507D7D 1
0700E850D28319CDB10F0D31BBFB911E9B1 2
07EA3763DDA9C2524CF7A94914230350490 1
0860CADB9DAF33D42BB85FBF321DC09

We can loop through the generator object.